In [3]:
import numpy as np
import pandas as pd

kospi200 = np.load('kospi200.npy')
samsung = np.load('samsung.npy')

print(kospi200)
print(samsung)
print(kospi200.shape)
print(samsung.shape)


def split_xy5(dataset, time_steps, y_column):
    x, y = list(), list()
    for i in range(len(dataset)):
        x_end_number = i + time_steps
        y_end_number = x_end_number + y_column # 수정

        if y_end_number > len(dataset):  # 수정
            break
        tmp_x = dataset[i:x_end_number, :]  # 수정
        tmp_y = dataset[x_end_number:y_end_number, 1]    # 수정
        x.append(tmp_x)
        y.append(tmp_y)
    return np.array(x), np.array(y)
x1, y1 = split_xy5(samsung, 5, 1) 
x2, y2 = split_xy5(kospi200, 5, 1) 
print(x2[0,:], "\n", y2[0])
print(x2.shape)
print(y2.shape)

# 데이터 셋 나누기
from sklearn.model_selection import train_test_split
# from sklearn.model_selection import cross_val_score
x1_train, x1_test, y1_train, y1_test = train_test_split(
    x1, y1, random_state=1, test_size = 0.3)
x2_train, x2_test, y2_train, y2_test = train_test_split(
    x2, y2, random_state=2, test_size = 0.3)

print(x2_train.shape)
print(x2_test.shape)
print(y2_train.shape)
print(y2_test.shape)

x1_train = np.reshape(x1_train,
    (x1_train.shape[0], x1_train.shape[1] * x1_train.shape[2]))
x1_test = np.reshape(x1_test,
    (x1_test.shape[0], x1_test.shape[1] * x1_test.shape[2]))
x2_train = np.reshape(x2_train,
    (x2_train.shape[0], x2_train.shape[1] * x2_train.shape[2]))
x2_test = np.reshape(x2_test,
    (x2_test.shape[0], x2_test.shape[1] * x2_test.shape[2]))
print(x2_train.shape)
print(x2_test.shape)


#### 데이터 전처리 #####
from sklearn.preprocessing import StandardScaler
scaler1 = StandardScaler()
scaler1.fit(x1_train)
x1_train_scaled = scaler1.transform(x1_train)
x1_test_scaled = scaler1.transform(x1_test)
scaler2 = StandardScaler()
scaler2.fit(x2_train)
x2_train_scaled = scaler2.transform(x2_train)
x2_test_scaled = scaler2.transform(x2_test)
print(x2_train_scaled[0, :])

[[1.000e+00 0.000e+00]
 [2.000e+00 0.000e+00]
 [3.000e+00 0.000e+00]
 ...
 [1.094e+03 2.000e-01]
 [1.095e+03 0.000e+00]
 [1.096e+03 0.000e+00]]
[[1.000e+00 9.063e+00]
 [2.000e+00 9.104e+00]
 [3.000e+00 3.198e+00]
 ...
 [1.094e+03 6.961e+00]
 [1.095e+03 6.997e+00]
 [1.096e+03 7.022e+00]]
(1096, 2)
(1096, 2)
[[1. 0.]
 [2. 0.]
 [3. 0.]
 [4. 0.]
 [5. 0.]] 
 [0.]
(1091, 5, 2)
(1091, 1)
(763, 5, 2)
(328, 5, 2)
(763, 1)
(328, 1)
(763, 10)
(328, 10)
[ 0.33483417 -0.32345121  0.33483417 -0.30786402  0.33483417 -0.29623321
  0.33483417 -0.30572726  0.33483417 -0.30558955]


In [4]:
x1_train_scaled = np.reshape(x1_train_scaled,
    (x1_train_scaled.shape[0], 5, 2))
x1_test_scaled = np.reshape(x1_test_scaled,
    (x1_test_scaled.shape[0], 5, 2))
x2_train_scaled = np.reshape(x2_train_scaled,
    (x2_train_scaled.shape[0], 5, 2))
x2_test_scaled = np.reshape(x2_test_scaled,
    (x2_test_scaled.shape[0], 5, 2))
print(x2_train_scaled.shape)
print(x2_test_scaled.shape)

(763, 5, 2)
(328, 5, 2)


In [6]:
from keras.models import Model
from keras.layers import Dense, Input, LSTM

# 모델구성
input1 = Input(shape=(5, 2))
dense1 = LSTM(64)(input1)
dense1 = Dense(32)(dense1)
dense1 = Dense(32)(dense1)
output1 = Dense(32)(dense1)

input2 = Input(shape=(5, 2))
dense2 = LSTM(64)(input2)
dense2 = Dense(64)(dense2)
dense2 = Dense(64)(dense2)
dense2 = Dense(64)(dense2)
output2 = Dense(32)(dense2)

from tensorflow.keras.layers import concatenate

merge = concatenate([output1, output2])
output3 = Dense(1)(merge)

model = Model(inputs=[input1, input2],
              outputs = output3 )


model.compile(loss='mse', optimizer='adam', metrics=['mse'])

from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience=20)
model.fit([x1_train_scaled, x2_train_scaled], y1_train, validation_split=0.2, 
          verbose=1, batch_size=1, epochs=100, 
          callbacks=[early_stopping])

loss, mse = model.evaluate([x1_test_scaled, x2_test_scaled], y1_test, batch_size=1)
print('loss : ', loss)
print('mse : ', mse)

y1_pred = model.predict([x1_test_scaled, x2_test_scaled])

for i in range(5):
    print('종가 : ', y1_test[i], '/ 예측가 : ', y1_pred[i])



Epoch 1/100
610/610 [==============================] - 3s 3ms/step - loss: 21635.1797 - mse: 21635.1797 - val_loss: 59102.2422 - val_mse: 59102.2422
Epoch 2/100
610/610 [==============================] - 1s 2ms/step - loss: 15509.0625 - mse: 15509.0625 - val_loss: 56335.9492 - val_mse: 56335.9492
Epoch 3/100
610/610 [==============================] - 1s 2ms/step - loss: 13532.7373 - mse: 13532.7373 - val_loss: 53471.4805 - val_mse: 53471.4805
Epoch 4/100
610/610 [==============================] - 1s 2ms/step - loss: 12603.9951 - mse: 12603.9951 - val_loss: 56208.2344 - val_mse: 56208.2344
Epoch 5/100
610/610 [==============================] - 1s 2ms/step - loss: 11350.8330 - mse: 11350.8330 - val_loss: 61377.9922 - val_mse: 61377.9922
Epoch 6/100
610/610 [==============================] - 1s 2ms/step - loss: 10159.1113 - mse: 10159.1113 - val_loss: 51532.8906 - val_mse: 51532.8906
Epoch 7/100
610/610 [==============================] - 1s 2ms/step - loss: 11551.0986 - mse: 11551.0986 - 